# 1. Import dependencies

In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:12.0
Episode:2 Score:17.0
Episode:3 Score:16.0
Episode:4 Score:20.0
Episode:5 Score:57.0


In [5]:
env.close()

In [6]:
env.action_space.sample()#0 sol 1 sağ

1

In [7]:
env.observation_space.sample()#araba konumu, araba hızı, kutup açısı, kutup açısal hızı

array([-7.3426801e-01,  1.0487342e+38, -1.2317792e-01,  1.9950567e+38],
      dtype=float32)

# 3. Train an RL Model

In [8]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cuda device


In [10]:
model.learn(total_timesteps = 20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 276  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 244         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008210543 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00112     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.43        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0159     |
|    value_loss           | 58.7        |
-----------------------------------------
---

# 4. Save and Reload Model

In [11]:
PPO_path = os.path.join(save_path, 'PPO_model')

In [12]:
model.save(PPO_path)

In [13]:
#model = PPO.load(PPO_path, env=env)

# 5. Evaluation

In [14]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\tolga\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [15]:
env.close()

# 6. Test Model

In [16]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [17]:
env.close()

# 7. Viewing Logs in Tensorboard

In [20]:
#!tensorboard --logdir=Training/Logs/PPO_1

# 8. Adding a callback to the training Stage

In [21]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [22]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [23]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [24]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 322  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 350         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008683635 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00204     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.27        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0159     |
|    value_loss           | 52.5        |
-----------------------------------------
---

In [25]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [26]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [27]:
env.close()

# 9. Changing Policies

In [28]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [29]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cuda device


In [30]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 302  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 279         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013561703 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00237    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.38        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0249     |
|    value_loss           | 16.1        |
-----------------------------------------
----------------------------------

# 10. Using an Alternate Algorithm

In [31]:
from stable_baselines3 import DQN

In [32]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [33]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.973    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4689     |
|    time_elapsed     | 0        |
|    total_timesteps  | 56       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.937    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6068     |
|    time_elapsed     | 0        |
|    total_timesteps  | 133      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.902    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6660     |
|    time_elapsed     | 0        |
|    total_timesteps  | 206      |
----------------------------------
------------------------

In [34]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [35]:
model.save(dqn_path)

In [36]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.4, 0.8)

In [37]:
env.close()